# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = 'cities.csv'


city_weather_df = pd.read_csv(weather_csv_file, index_col=0)


city_weather_df.head()


,Cloudiness,Country,Humidity,Lat,Lng,Max Temperature,Wind Speed,City
0,15,RU,66,64.38,-173.30,40.93,18.03,provideniya
1,85,RU,30,70.63,147.92,67.48,12.75,chokurdakh
2,20,AR,59,-54.80,-68.30,35.60,18.34,ushuaia
3,81,AU,58,-33.65,115.33,63.00,17.63,busselton
4,49,CN,33,49.20,119.70,77.83,7.96,hailar


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
city_weather_df = city_weather_df.dropna()
humidity = city_weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = city_weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
city_weather_df

,Cloudiness,Country,Humidity,Lat,Lng,Max Temperature,Wind Speed,City
0,15,RU,66,64.38,-173.30,40.93,18.03,provideniya
1,85,RU,30,70.63,147.92,67.48,12.75,chokurdakh
2,20,AR,59,-54.80,-68.30,35.60,18.34,ushuaia
3,81,AU,58,-33.65,115.33,63.00,17.63,busselton
4,49,CN,33,49.20,119.70,77.83,7.96,hailar
...,...,...,...,...,...,...,...,...
534,15,GB,82,51.80,-4.97,57.20,29.97,avera
535,100,IN,75,18.77,84.42,84.90,10.02,gizo
536,99,RU,71,56.05,99.51,73.09,2.82,taltal
537,6,NZ,70,-41.08,173.00,57.99,3.00,srednekolymsk


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
updated_weather_df = city_weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
updated_weather_df = updated_weather_df.loc[(updated_weather_df["Max Temperature"] < 80) & (updated_weather_df["Max Temperature"] > 70)]

# Wind speed less than 10 mph.
updated_weather_df = updated_weather_df.loc[updated_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
updated_weather_df = updated_weather_df.loc[updated_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
updated_weather_df = updated_weather_df.dropna()

updated_weather_df 

,Cloudiness,Country,Humidity,Lat,Lng,Max Temperature,Wind Speed,City
26,0,AU,27,-20.73,139.50,73.40,5.82,port elizabeth
44,0,RU,40,53.73,119.77,75.02,3.76,port alfred
82,0,RU,46,61.28,108.01,76.46,5.61,tuktoyaktuk
110,0,CN,33,40.28,97.20,71.82,4.76,sorong
119,0,LY,50,29.13,15.95,73.17,6.42,yumen
127,0,CN,54,37.15,122.38,78.96,7.27,norsup
178,0,IQ,34,33.34,44.40,78.80,3.36,itatiba
227,0,GA,89,-1.57,9.26,70.20,4.07,bojnurd
244,0,LY,85,32.53,20.58,70.92,5.68,saskylakh
306,0,MN,25,48.01,91.64,73.78,2.42,weligama


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotels_df = updated_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotels_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotels_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel is ibis Styles Mt Isa Verona.
------------
Closest hotel is Yalta Inn.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Closest hotel is Alrwasi Hotel.
------------
Closest hotel is GreenTree Hotel Rongcheng Bus Station.
------------
Closest hotel is Baghdad Hotel.
------------
Closest hotel is Hotel Olako.
------------
Closest hotel is استراحة لاقروتا.
------------
Closest hotel is Hotel Khovd.
------------
Closest hotel is JJ Inns - Yinchuan New Moon Square Branch.
------------
Closest hotel is Requinte Hotel.
------------
Closest hotel is Aracan Hotel.
------------
Closest hotel is רגע בנחת.
------------
Closest hotel is Jiugang Hotel - Jiayuguan.
------------


In [7]:
hotels_df

,Cloudiness,Country,Humidity,Lat,Lng,Max Temperature,Wind Speed,City,Hotel Name
26,0,AU,27,-20.73,139.50,73.40,5.82,port elizabeth,ibis Styles Mt Isa Verona
44,0,RU,40,53.73,119.77,75.02,3.76,port alfred,Yalta Inn
82,0,RU,46,61.28,108.01,76.46,5.61,tuktoyaktuk,NaN
110,0,CN,33,40.28,97.20,71.82,4.76,sorong,NaN
119,0,LY,50,29.13,15.95,73.17,6.42,yumen,Alrwasi Hotel
127,0,CN,54,37.15,122.38,78.96,7.27,norsup,GreenTree Hotel Rongcheng Bus Station
178,0,IQ,34,33.34,44.40,78.80,3.36,itatiba,Baghdad Hotel
227,0,GA,89,-1.57,9.26,70.20,4.07,bojnurd,Hotel Olako
244,0,LY,85,32.53,20.58,70.92,5.68,saskylakh,استراحة لاقروتا
306,0,MN,25,48.01,91.64,73.78,2.42,weligama,Hotel Khovd


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_locations = [
    (-34.0, -59.166672),
    (-32.23333, -64.433327),
    (40.166672, 44.133331),
    (51.216671, 5.0833302),
    (51.333328, 4.25)
]

# fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))